<sub>Developed by SeongKu Kang, August 2025 — Do not distribute</sub>

# Product Category Classification with Label GNN

This notebook builds on the previous assignment **P2c: train-with-BERT-embedding-limited-label**.  
The setup is similar, but here we take one step further by **leveraging label structure information**.

---

### Recall: Label Embedding (Inner Product) Classifier
- Each **data instance** (e.g., a product description) is projected into a hidden space.  
- Each **label** has its own embedding vector.  
- Classification is performed via an **inner product** between the projected instance and label embeddings.  

Formally:
$$
\text{logits}(x) = \text{Proj}(x) \cdot E^T
$$

where  
- $\text{Proj}(x)$ is the projected input representation,  
- $E$ is the matrix of label embeddings.  

Originally, label embeddings $E$ were initialized with **BERT mean-pooled representations** of the label names.  
One small difference from the previous P2 assignment is that we now use only leaf-level categories for the label embeddings.  

---

### What’s New Here?
- In addition to BERT initialization, we are now given **a graph of label relationships**, derived from the category hierarchy.  
  - Example: if two categories share the same parent (e.g., *“iPhone Cases”* and *“iPhone Chargers”* under *“iPhone Accessories”*), we connect them with an edge.  
  - We provide the **normalized adjacency matrix**, so your main task is to **implement the GNN part**.

- To enrich the label embeddings, we propagate information over this label graph using a **Graph Neural Network (GNN)**:
  $$
  E' = \text{GNN}(E, A_{\text{label}})
  $$
  where $E$ are the initial label embeddings and $A_{\text{label}}$ is the adjacency between labels.  

- Final classification then uses these **graph-enriched label embeddings**:
  $$
  \text{logits}(x) = \text{Proj}(x) \cdot (E')^T
  $$

In [ ]:
import json
from tqdm import tqdm
from pathlib import Path
from utils import * 
import copy

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split

device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
# Default paths
ROOT = Path("dataset") # Root dataset directory
CORPUS_PATH = ROOT / "corpus.jsonl" # Product corpus file (JSON Lines): Each line contains a product ID and its associated text description.
EMB_PATH = ROOT / "corpus_bert_mean.pt"

# Task 1: Product category classification
LABEL_MAP_PATH = ROOT / "category_classification" 
LABEL2ID_PATH = LABEL_MAP_PATH / "label2labelid.json" 
ID2LABEL_PATH = LABEL_MAP_PATH / "labelid2label.json" 
PID2LABEL_TRAIN_LIMITED_PATH = LABEL_MAP_PATH / "pid2labelids_train_limited.json" 
PID2LABEL_TEST_PATH = LABEL_MAP_PATH / "pid2labelids_test.json" 
LABEL_EMB_PATH = LABEL_MAP_PATH / "category_labels_leaf_bert_mean.pt"

In [ ]:
pid2text = load_corpus(CORPUS_PATH) # load corpus

label2id = load_json(LABEL2ID_PATH)
id2label = load_json(ID2LABEL_PATH)
pid2label_train_limited = load_json(PID2LABEL_TRAIN_LIMITED_PATH)
pid2label_test = load_json(PID2LABEL_TEST_PATH)

# loading pre-trained embeddings
corpus_data = torch.load(EMB_PATH)  # {"ids": [...], "embeddings": Tensor}
pid_list = corpus_data["ids"]
pid2idx = {pid: i for i, pid in enumerate(pid_list)}
embeddings = corpus_data["embeddings"]

label_data = torch.load(LABEL_EMB_PATH)
label_emb = label_data["embeddings"].to(device)

In [ ]:
class ProductCategoryEmbeddingDataset(Dataset):
    def __init__(self, pid2label, pid2idx, embeddings):
        self.pids = list(pid2label.keys())
        self.labels = [pid2label[pid] for pid in self.pids]
        self.indices = [pid2idx[pid] for pid in self.pids]
        self.embeddings = embeddings

    def __len__(self):
        return len(self.pids)

    def __getitem__(self, idx):
        emb = self.embeddings[self.indices[idx]]
        label = torch.tensor(self.labels[idx], dtype=torch.long)
        return {"X": emb, "y": label}

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

def evaluate(model, dataloader, device="cpu"):
    model.eval()
    all_preds, all_labels = [], []

    with torch.no_grad():
        for batch in dataloader:
            X = batch["X"].to(device)
            y = batch["y"].to(device)
            logits = model(X)
            preds = torch.argmax(logits, dim=1)
            all_preds.extend(preds.cpu().tolist())
            all_labels.extend(y.cpu().tolist())

    acc = accuracy_score(all_labels, all_preds)
    f1_macro = f1_score(all_labels, all_preds, average="macro", zero_division=0)

    return {"accuracy": acc, "f1_macro": f1_macro}

In [ ]:
# Build test dataset and dataloader from precomputed embeddings
test_dataset = ProductCategoryEmbeddingDataset(pid2label_test, pid2idx, embeddings)
test_loader = DataLoader(test_dataset, batch_size=64)

# Model dimensions
input_dim = embeddings.shape[1]   # Size of embedding vector (feature dimension)
num_classes = len(label2id)       # Number of category classes

# Keep track of product IDs
all_pids = set(pid_list)
test_pids = set(pid2label_test.keys())

In [ ]:
# Build dataset with limited labeled products
train_dataset = ProductCategoryEmbeddingDataset(pid2label_train_limited, pid2idx, embeddings)

# Split into train/validation sets (80% / 20%)
val_ratio = 0.2
val_size = int(len(train_dataset) * val_ratio)
train_size = len(train_dataset) - val_size

train_split, val_split = random_split(train_dataset, [train_size, val_size])

# DataLoaders for training and validation
train_loader = DataLoader(train_split, batch_size=32, shuffle=True)
val_loader = DataLoader(val_split, batch_size=64)

In [ ]:
# Find products without labels
used_pids = set(pid2label_train_limited.keys())
unlabeled_pids = all_pids - used_pids

# Ratio of unlabeled products in the whole corpus
unlabeled_ratio = len(unlabeled_pids) / len(all_pids)

print(f"[Unlabeled] {len(unlabeled_pids)} / {len(all_pids)} = {unlabeled_ratio:.2%}")

In [ ]:
results_dict = {'valid':{}, 'test': {}}

## [Part A]: revisit Classification with label embedding

This part is identical to the previous assignment P2c: train-with-BERT-embedding-limited-label.

In [ ]:
# Classifier that uses label embeddings to make predictions
class InnerProductClassifier(nn.Module):
    def __init__(self, input_dim, label_embeddings, trainable_label_emb=True):
        super().__init__()
        # Project input features into the same dimension as label embeddings
        self.proj = nn.Linear(input_dim, label_embeddings.size(1))

        if trainable_label_emb:
            # Label embeddings are trainable parameters
            self.label_emb = nn.Parameter(label_embeddings.clone())
        else:
            # Label embeddings are fixed (not updated during training)
            self.register_buffer("label_emb", label_embeddings.clone())

    def forward(self, x):
        # Project input feature vectors
        x_proj = self.proj(x)
        # Compute logits as similarity with each label embedding
        logits = torch.matmul(x_proj, self.label_emb.T)
        return logits

In [ ]:
model = InnerProductClassifier(input_dim, label_emb).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-4)

In [ ]:
best_val_acc = -1
best_model_state = None
patience = 5
patience_counter = 0

val_acc_list = []
test_acc_list = []

EPOCHS = 500

for epoch in range(1, EPOCHS + 1):
    model.train()
    total_loss = 0

    for batch in tqdm(train_loader, desc=f"Epoch {epoch}"):
        X = batch["X"].to(device)
        y = batch["y"].to(device)
        logits = model(X)
        loss = F.cross_entropy(logits, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"[Epoch {epoch}] Train Loss: {avg_loss:.4f}")

    # === Validation ===
    val_result = evaluate(model, val_loader, device=device)
    val_acc = val_result["accuracy"]
    val_acc_list.append(val_acc)

    is_improved = val_acc > best_val_acc
    print_eval_result(val_result, stage="val", is_improved=is_improved)

    # === Test ===
    test_result = evaluate(model, test_loader, device=device)
    test_acc = test_result["accuracy"]
    test_acc_list.append(test_acc)
    print_eval_result(test_result, stage="test")

    # === Update best model ===
    if is_improved:
        best_val_acc = val_acc
        best_model_state = copy.deepcopy(model.state_dict())
        patience_counter = 0
    else:
        patience_counter += 1

    # === Early stopping ===
    if patience_counter >= patience:
        print(f"[Early Stopping] No improvement for {patience} consecutive epochs.")
        break

In [ ]:
model.load_state_dict(best_model_state)
final_test_result = evaluate(model, test_loader, device=device)
print_eval_result(final_test_result, stage="final_test")

results_dict['valid']['label_train'] = val_acc_list[:]
results_dict['test']['label_train'] = test_acc_list[:]

## [Part B] Your Task: Implement Label GCN and GCN-Enhanced Classifier

In this exercise, you will implement a **Graph Convolutional Network (GCN)** for label embeddings, and use it to build a **GCN-enhanced classifier** for product category classification.

---

### 1. Label GCN (`LabelGCN`)
- Input:
  - Initial label embeddings $E \in \mathbb{R}^{\text{num\_labels} \times d}$
  - Normalized adjacency matrix $A_{\text{hat}} \in \mathbb{R}^{\text{num\_labels} \times \text{num\_labels}}$
- For each layer:
  1. **Message passing:** $H \leftarrow A_{\text{hat}} H$
  2. **Linear transform:** $H \leftarrow HW$
  3. **Activation + Dropout (optional):** Apply ReLU and Dropout, except for the last layer
- Output: Refined label embeddings $E' \in \mathbb{R}^{\text{num\_labels} \times d}$

---

### 2. GCN-Enhanced Classifier (`GCNEnhancedClassifier`)
- Goal: Classify a document by comparing its embedding with **graph-enriched label embeddings**.
- Steps:
  1. **Projection:** Project input document embeddings into the same space as label embeddings.
  2. **Label GCN:** Refine label embeddings using the GCN.
  3. **Inner Product Scoring:** Compute logits by inner product between projected documents and refined label embeddings:
     $$
     \text{logits}(x) = \text{Proj}(x) \cdot (E')^T
     $$
- Input: 
  - $x \in \mathbb{R}^{\text{batch\_size} \times \text{input\_dim}}$ (document embeddings)
  - Initial label embeddings $E$
  - Label adjacency $A_{\text{hat}}$
- Output: 
  - $\text{logits}(x) \in \mathbb{R}^{\text{batch\_size} \times \text{num\_labels}}$

---
You need to implement:
1. **LabelGCN**
   - Define learnable weights for each layer
   - Implement message passing and transformations
   - Add ReLU + Dropout (except the last layer)

2. **GCNEnhancedClassifier**
   - Define projection layer for documents
   - Apply GCN to refine label embeddings
   - Compute logits with inner product

👉 Start from the provided skeleton code and fill in the missing parts (`# TODO`).

---

### ✅ What to submit
- No Kaggle submission this time.  
- Submit **only your completed notebook** via LMS.

In [ ]:
# Precomputed normalized adjacency matrix for label graph
# (You don't need to worry about how it's generated — just use A_hat directly)
A_hat = build_leaf_adj(id2label, label2id)

In [ ]:
# ==========================================================
# Your Task: Implement Label GCN and GCN-Enhanced Classifier
# ==========================================================

class LabelGCN(nn.Module):
    """
    Multi-layer Graph Convolutional Network (GCN) encoder for label embeddings.

    Each layer should perform the following steps:
        1. Aggregate neighbor embeddings: H <- A_hat @ H
        2. Linear transformation: H <- H @ W
        3. (Optional) Apply ReLU and Dropout (skip for the last layer)

    Args:
        emb_dim (int): Dimension of label embeddings.
        num_layers (int): Number of GCN layers.
        dropout (float): Dropout probability.
    """
    def __init__(self, emb_dim, num_layers=1, dropout=0.5):
        super().__init__()
        # TODO: Define learnable weight matrices (list of emb_dim x emb_dim parameters)
        # Hint: Use nn.ParameterList and Xavier uniform initialization
        raise NotImplementedError

    def forward(self, H, A_hat):
        """
        Args:
            H (torch.Tensor): Initial label embeddings, shape (num_labels, emb_dim).
            A_hat (torch.Tensor): Normalized adjacency matrix, shape (num_labels, num_labels).

        Returns:
            torch.Tensor: Updated label embeddings, shape (num_labels, emb_dim).
        """
        # TODO: Implement multi-layer GCN
        # for each layer:
        #   1) propagate messages: H = A_hat @ H
        #   2) linear transform: H = H @ W
        #   3) if not last layer: apply ReLU + Dropout
        raise NotImplementedError


class GCNEnhancedClassifier(nn.Module):
    """
    Classifier that combines:
      - Document representations projected into label space
      - Label embeddings refined by a GCN over the label graph

    Args:
        input_dim (int): Dimension of input document embeddings.
        label_init_emb (torch.Tensor): Initial label embeddings, shape (num_labels, emb_dim).
        A_hat (torch.Tensor): Normalized adjacency matrix of labels, shape (num_labels, num_labels).
        num_layers (int): Number of GCN layers.
        dropout (float): Dropout probability.
    """
    def __init__(self, input_dim, label_init_emb, A_hat, num_layers=1, dropout=0.5):
        super().__init__()
        # TODO: 
        # 1. Define projection layer (input_dim -> emb_dim)
        # 2. Define GCN encoder for label embeddings
        # 3. Make label_init_emb trainable (nn.Parameter)
        # 4. Register adjacency matrix (use register_buffer)
        raise NotImplementedError

    def forward(self, x):
        """
        Args:
            x (torch.Tensor): Input embeddings for documents, shape (batch_size, input_dim).

        Returns:
            torch.Tensor: Logits for classification, shape (batch_size, num_labels).
        """
        # TODO:
        # 1. Refine label embeddings using GCN
        # 2. Project input x into label embedding space (+ dropout)
        # 3. Compute similarity (inner product) between x_proj and label_emb
        raise NotImplementedError

In [ ]:
model = GCNEnhancedClassifier(embeddings.size(1), label_emb, A_hat.to(device), num_layers=1).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-4)

In [ ]:
best_val_acc = -1
best_model_state = None
patience = 5
patience_counter = 0

val_acc_list = []
test_acc_list = []

EPOCHS = 500

for epoch in range(1, EPOCHS + 1):
    model.train()
    total_loss = 0

    for batch in tqdm(train_loader, desc=f"Epoch {epoch}"):
        X = batch["X"].to(device)
        y = batch["y"].to(device)
        logits = model(X)
        loss = F.cross_entropy(logits, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"[Epoch {epoch}] Train Loss: {avg_loss:.4f}")

    # === Validation ===
    val_result = evaluate(model, val_loader, device=device)
    val_acc = val_result["accuracy"]
    val_acc_list.append(val_acc)

    is_improved = val_acc > best_val_acc
    print_eval_result(val_result, stage="val", is_improved=is_improved)

    # === Test ===
    test_result = evaluate(model, test_loader, device=device)
    test_acc = test_result["accuracy"]
    test_acc_list.append(test_acc)
    print_eval_result(test_result, stage="test")

    # === Update best model ===
    if is_improved:
        best_val_acc = val_acc
        best_model_state = copy.deepcopy(model.state_dict())
        patience_counter = 0
    else:
        patience_counter += 1

    # === Early stopping ===
    if patience_counter >= patience:
        print(f"[Early Stopping] No improvement for {patience} consecutive epochs.")
        break

In [ ]:
# model.load_state_dict(best_model_state)
final_test_result = evaluate(model, test_loader, device=device)
print_eval_result(final_test_result, stage="final_test")

results_dict['valid']['label_GNN_train'] = val_acc_list[:]
results_dict['test']['label_GNN_train'] = test_acc_list[:]

In [ ]:
plot_results(results_dict, split="valid")
plot_results(results_dict, split="valid", metric='F1-macro')